In [14]:
import sys
sys.path.append('/Users/natalia.murycheva/PycharmProjects/gitCommitMessageCollector')

In [15]:
import re
import itertools
from typing import List
from pathlib import Path
from common_dataset.logs import COMMON_SEP
from code2seq_dataset.global_vars import Message
from code2seq_dataset.common import split_commit_message
from code2seq_dataset.info_classes import FullLogLine, CommitLogLine
from message_analyzing.utils import messages_frequency, messages_vs_author
from message_analyzing.utils import get_messages_from_log, get_messages_from_json

# Camel

In [3]:
git_dir_name: str = 'camel'
raw_dir: Path = Path.cwd().parent.parent.joinpath('data').joinpath('raw_data').joinpath(git_dir_name)
commits_log: Path = raw_dir.joinpath('commits.log')
processed_dir: Path = Path.cwd().parent.parent.joinpath('data').joinpath('processed_data').joinpath(git_dir_name)
filtered_json = Path = processed_dir.joinpath('diff_filtered.json')

In [6]:
all_messages: List[Message] = get_messages_from_log(commits_log)
filterd_messages: List[Message] = get_messages_from_json(filtered_json)

## Messages counts

In [7]:
dict(itertools.islice(messages_frequency(all_messages).items(), 0, 20))

{466: ['polished'],
 462: ['regen'],
 377: ['fixed cs'],
 191: ['fixed test'],
 188: ['component docs'],
 185: ['camel <num> fixed cs'],
 119: ['camel <num> polished'],
 103: ['camel <num> added unit test'],
 98: ['upgrade chronicle wire to version <num> <num>'],
 96: ['removed unused imports'],
 89: ['upgrade chronicle bytes to version <num> <num>'],
 74: ['added test based on user forum issue',
  'camel <num> more components include documentation'],
 72: ['upgrade chronicle core to version <num> <num>'],
 65: ['regen docs'],
 60: ['polish'],
 56: ['upgrade chronicle queue to version <num> <num>',
  'fixed cs this closes <num>'],
 51: ['upgrade kubernetes client to version <num> <num>',
  'upgrade async http client to version <num> <num>',
  'upgrade chronicle network to version <num> <num>'],
 50: ['camel <num> regen'],
 49: ['fix cs', 'added unit test based on user forum issue'],
 47: ['camel <num>']}

In [8]:
dict(itertools.islice(messages_frequency(all_messages).items(), 0, 60))

{466: ['polished'],
 462: ['regen'],
 377: ['fixed cs'],
 191: ['fixed test'],
 188: ['component docs'],
 185: ['camel <num> fixed cs'],
 119: ['camel <num> polished'],
 103: ['camel <num> added unit test'],
 98: ['upgrade chronicle wire to version <num> <num>'],
 96: ['removed unused imports'],
 89: ['upgrade chronicle bytes to version <num> <num>'],
 74: ['added test based on user forum issue',
  'camel <num> more components include documentation'],
 72: ['upgrade chronicle core to version <num> <num>'],
 65: ['regen docs'],
 60: ['polish'],
 56: ['upgrade chronicle queue to version <num> <num>',
  'fixed cs this closes <num>'],
 51: ['upgrade kubernetes client to version <num> <num>',
  'upgrade async http client to version <num> <num>',
  'upgrade chronicle network to version <num> <num>'],
 50: ['camel <num> regen'],
 49: ['fix cs', 'added unit test based on user forum issue'],
 47: ['camel <num>'],
 44: ['fixed tests', 'fixed the cs errors'],
 43: ['upgrade infinispan to version 

- Много сообщений, которые имеют следующий паттерн: 
    `upgrade <something> to version <num>`
- Не особо видно `rollback` или `merge`

Посмотрим, какие файлы меняют в таких коммитах (`upgrade <something> to version <num>`)

In [17]:
changed_files_log: Path = output_dir.joinpath("changed_java_files.log")
    
i = 0
with open(changed_files_log, 'r') as f:
    for line in f:
        changed_files_log_line = FullLogLine.parse_from_line(line, separator=COMMON_SEP)
        clear_message = " ".join(split_commit_message(changed_files_log_line.message))
        if re.findall(r'upgrade [\w ]+ to version', clear_message):
            i += 1
            print(f'file: {changed_files_log_line.file}, \n'
                  f'old_blob: {changed_files_log_line.old_blob}, \n'
                  f'new_blob: {changed_files_log_line.new_blob}, \n'
                  f'message: {changed_files_log_line.message}\n')
            if i > 30: break

file: platforms/spring-boot/components-starter/camel-infinispan-starter/src/main/java/org/apache/camel/component/infinispan/springboot/InfinispanComponentAutoConfiguration.java, 
old_blob: 3396e448a06ff432d33bd46ab95d05bbfc9e85d6, 
new_blob: 9b17bf69646880dee7173af1a7f9dca03bb00c97, 
message: Upgrade Infinispan to version 9.4.10.Final

file: platforms/spring-boot/components-starter/camel-infinispan-starter/src/main/java/org/apache/camel/component/infinispan/springboot/InfinispanComponentConfiguration.java, 
old_blob: 79e5995af5c5854b742a234fb407c89cc6dd3316, 
new_blob: c8c4ea3c2a8c47b3fc697db057bc0a9a3c0e2ed2, 
message: Upgrade Infinispan to version 9.4.10.Final

file: tests/camel-itest-karaf/src/test/java/org/apache/camel/itest/karaf/CamelIgniteTest.java, 
old_blob: 80b869a9a1dfad8bd9c6e82630df24b1e54aea8a, 
new_blob: 473368a6916887557786dd09ef9c76b72890035a, 
message: Upgrade Ignite to version 2.7.0

file: components/camel-infinispan/src/test/java/org/apache/camel/component/infinispa

Так как я фильтровала и записывала только закомиченные джава файлы, то тут только джава файлы.

### Нельзя сказать, что в этих коммитах однотипные изменения, однако большое количество шаблонных сообщений может влиять на величину bleu-score-a

что увидела:
- изменение вида многострочных комментариев
- замена в тестах строковой переменной, хранящей номер версии
- изменение импортов
- удаление методов
- добавление дженериков
- измение интерфейса, от которого наследуется класс

------

Посмотрим, какой процент от всего датасета составляют коммиты с подобным сообщением:

In [9]:
total_commits_number: int = 0
commits_with_templates_number: int = 0
for message in all_messages:
    total_commits_number += 1
    message = Message(" ".join(split_commit_message(message)))
    if re.findall(r'upgrade [\w ]+ to version', message):
        commits_with_templates_number += 1

print(commits_with_templates_number)
print((commits_with_templates_number / total_commits_number) * 100)

2771
7.036209435782845


Получили довольно большой процент, 7%

(для сравнения, в NMT статье мусора и шаблонных сообщений было около 16% и bleu-score там был в два раза выше настоящего)

## Посмотрим на отфильтрованные сообщения
_All commits number is 39382._

_Commits that changed java files - 27374._

_After filtering we have 25022 commits._

In [11]:
dict(itertools.islice(messages_frequency(filterd_messages).items(), 0, 60))

{553: ['fixed cs'],
 460: ['polished'],
 212: ['fixed test'],
 163: ['component docs'],
 112: ['added unit test'],
 99: ['removed unused imports'],
 87: ['regen'],
 75: ['added test based on user forum issue'],
 74: ['more components include documentation'],
 64: ['polish'],
 63: ['fixed cs this closes <num>'],
 61: ['fixed the cs errors'],
 54: ['fixed tests'],
 53: ['fix cs'],
 50: ['added unit test based on user forum issue'],
 39: ['polished javadoc', 'add more eip as specialized mbeans'],
 37: ['fixed test on other boxes'],
 36: ['mr <num> added more unit tests'],
 31: ['fixed unit test'],
 30: ['add property binding support to make it convenient to configure components and whatnot',
  'circuit breaker eip that is using hystrix',
  'optimized routing engine to reduce stack frames in use during routing work in progress'],
 28: ['fix checkstyle issues',
  'fixed cs error',
  'camel zipkin instrument camel work in progress'],
 26: ['fixed potential npe',
  'use awaitility in camel co

## Message vs. Author

In [23]:
dict(itertools.islice(messages_vs_author(commits_log).items(), 0, 30))

{405: [('Claus Ibsen', 'polished')],
 263: [('Claus Ibsen', 'regen')],
 256: [('Claus Ibsen', 'fixed cs')],
 191: [('Claus Ibsen', 'fixed test')],
 186: [('Claus Ibsen', 'component docs')],
 171: [('Andrea Cosentino', 'regen')],
 108: [('Claus Ibsen', 'camel <num> polished')],
 106: [('Andrea Cosentino', 'fixed cs')],
 104: [('Andrea Cosentino', 'camel <num> fixed cs')],
 100: [('Claus Ibsen', 'camel <num> added unit test')],
 98: [('Andrea Cosentino', 'upgrade chronicle wire to version <num> <num>')],
 90: [('Andrea Cosentino', 'removed unused imports')],
 89: [('Andrea Cosentino', 'upgrade chronicle bytes to version <num> <num>')],
 80: [('Claus Ibsen', 'camel <num> fixed cs')],
 74: [('Claus Ibsen', 'camel <num> more components include documentation')],
 72: [('Andrea Cosentino', 'upgrade chronicle core to version <num> <num>')],
 71: [('Claus Ibsen', 'added test based on user forum issue')],
 57: [('Babak Vahdat', 'polished')],
 56: [('Andrea Cosentino', 'upgrade chronicle queue to

# IntelliJ

In [16]:
git_dir_name: str = 'intellij'
output_dir: Path = Path.cwd().parent.parent.joinpath('data').joinpath("raw_data").joinpath(git_dir_name)
commits_log: Path = output_dir.joinpath("commits.log")
all_messages: List[Message] = get_messages_from_log(commits_log)

## Messages counts

In [17]:
dict(itertools.islice(messages_frequency(all_messages).items(), 0, 20))

{1847: ['cleanup'],
 267: ['cleanup formatting'],
 232: ['notnull'],
 130: ['cpp <num> integration with valgrind memory analyser'],
 129: ['updating versions of project dependencies jdk build u <num> b <num>'],
 111: ['fix testdata', 'fix compilation'],
 103: ['rename java to kt', 'cleanup warnings formatting'],
 102: ['cpp <num> google sanitizers support'],
 90: ['fix tests', 'ea <num>'],
 83: ['typo'],
 79: ['cpp <num> show cmake output messages while loading'],
 76: ['fix test data'],
 70: ['minor', 'ws <num> eap'],
 69: ['fix test'],
 67: ['ux <num> new search everywhere popup'],
 62: ['cleanup warnings'],
 58: ['fix npe'],
 54: ['fix typo'],
 53: ['not null'],
 50: ['cpp <num> perf support for clion']}

In [19]:
dict(itertools.islice(messages_frequency(all_messages).items(), 0, 60))

{1847: ['cleanup'],
 267: ['cleanup formatting'],
 232: ['notnull'],
 130: ['cpp <num> integration with valgrind memory analyser'],
 129: ['updating versions of project dependencies jdk build u <num> b <num>'],
 111: ['fix testdata', 'fix compilation'],
 103: ['rename java to kt', 'cleanup warnings formatting'],
 102: ['cpp <num> google sanitizers support'],
 90: ['fix tests', 'ea <num>'],
 83: ['typo'],
 79: ['cpp <num> show cmake output messages while loading'],
 76: ['fix test data'],
 70: ['minor', 'ws <num> eap'],
 69: ['fix test'],
 67: ['ux <num> new search everywhere popup'],
 62: ['cleanup warnings'],
 58: ['fix npe'],
 54: ['fix typo'],
 53: ['not null'],
 50: ['cpp <num> perf support for clion'],
 47: ['cpp <num> add option to change variable naming style'],
 46: ['npe', 'testdata for idea <num>'],
 44: ['simplify', 'idea project configuration the great module rename'],
 43: ['typescript more tests',
  'oc <num> cleanup',
  'merging android studio <num>',
  'make ig test lig

## Messages vs. Author

In [13]:
#dict(itertools.islice(messages_vs_author(commits_log).items(), 0, 35))